---
Proyecto del seguidor solar implementacion en codigo
---
Grupo #1
---

---
Librerias
---

In [7]:
from ipywidgets import interact, widgets
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime, timedelta
import numpy as np
from pytz import timezone
import math
from pysolar.solar import get_altitude, get_azimuth
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime, timedelta

---
Calculo de Angulos Pitch y Roll 
---

In [2]:
def obtenerPosicionSolar(
    latitud: float = -0.2105367,
    longitud: float = -78.491614,
    fecha: datetime = datetime.now(tz=timezone("America/Guayaquil")),
):
    """Calcula el azimut y la elevación para una posición geográfica y fecha.

    Parameters:
    latitud (float): Latitud del lugar.
    longitud (float): Longitud del alugar.
    fecha (datetime): Fecha y hora.

    Returns:
    float, float: Azimut (en grados), Elevación (en grados).
    """
    azimut = get_azimuth(latitud, longitud, fecha)
    elevacion = get_altitude(latitud, longitud, fecha)

    return azimut, elevacion


def calcularAngulosDeControl(azimut, elevacion):
    """Calcula los ángulos de control pitch y roll basados en el azimut y la elevación solar.

    Parameters:
    azimut (float): Azimut en grados.
    elevacion (float): Elevación en grados.

    Returns:
    float, float: Ángulo de pitch (en grados), Ángulo de roll (en grados).
    """
    theta = math.radians(elevacion)
    alpha = math.radians(azimut)

    # Fórmula del ángulo de cabeceo (pitch):
    # p = arcsin(-cos(θ) * cos(α))
    pitch = math.degrees(math.asin(-math.cos(theta) * math.cos(alpha)))

    # Fórmula del ángulo de balanceo (roll):
    # r = arccos(sin(θ) / cos(p))
    if math.cos(math.radians(pitch)) != 0:
        roll = math.degrees(math.acos(math.sin(theta) / math.cos(math.radians(pitch))))
    else:
        roll = 0

    return pitch, roll


if __name__ == "__main__":
    azimut, elevacion = obtenerPosicionSolar()

    pitch, roll = calcularAngulosDeControl(azimut, elevacion)

    print(f"Ángulo de Pitch: {pitch:.2f}°")
    print(f"Ángulo de Roll: {roll:.2f}°")

Ángulo de Pitch: -13.04°
Ángulo de Roll: 155.37°


C:\Users\agust\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pysolar\solartime.py:113: UserWarning: I don't know about leap seconds after 2023
  warnings.warn \


---
Interfaz Grafica
---

In [11]:
def spherical_to_cartesian(azimuths, elevations):
    """Convierte azimut y elevación a coordenadas 3D esféricas."""
    azimuths_rad = np.radians(azimuths)
    elevations_rad = np.radians(elevations)

    sun_x = np.cos(elevations_rad) * np.sin(azimuths_rad)
    sun_y = np.cos(elevations_rad) * np.cos(azimuths_rad)
    sun_z = np.sin(elevations_rad)

    return sun_x, sun_y, sun_z

def euler_to_cartesian(pitches, rolls):
    """Convierte pitch y roll a coordenadas 3D usando rotaciones de Euler."""
    pitches_rad = np.radians(pitches)
    rolls_rad = np.radians(rolls)

    panel_x = np.cos(pitches_rad) * np.sin(rolls_rad)
    panel_y = np.cos(pitches_rad) * np.cos(rolls_rad)
    panel_z = np.sin(pitches_rad)

    return panel_x, panel_y, panel_z

def plotSunAndPanelTrajectory(start_date, duration_hours, start_hour, latitude=-0.2105367, longitude=-78.491614):
    """Dibuja la trayectoria del sol y del panel solar de manera interactiva."""
    if start_date.tzinfo is None:
        start_date = timezone("America/Guayaquil").localize(start_date)

    end_date = start_date + timedelta(hours=duration_hours)
    times = [start_date + timedelta(minutes=10*i) for i in range(int((end_date - start_date).total_seconds() / 600))]
    
    azimuths, elevations, pitches, rolls = zip(*[
        (*obtenerPosicionSolar(latitude, longitude, time), *calcularAngulosDeControl(*obtenerPosicionSolar(latitude, longitude, time)))
        for time in times
    ])

    # Convertir azimut y elevación a coordenadas 3D
    sun_x, sun_y, sun_z = spherical_to_cartesian(azimuths, elevations)

    # Convertir pitch y roll a coordenadas 3D
    panel_x, panel_y, panel_z = euler_to_cartesian(pitches, rolls)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(sun_x, sun_y, sun_z, label='Trayectoria del Sol', color='yellow')
    ax.plot(panel_x, panel_y, panel_z, label='Orientación del Panel', color='red')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Trayectoria del Sol y Orientación del Panel Solar\nFecha: {start_date.strftime("%Y-%m-%d")}, Hora Inicial: {start_hour}:00, Duración: {duration_hours}h')
    ax.legend()
    plt.show()

def interactive_plot():
    """Crea una interfaz interactiva para ingresar la fecha, hora inicial y duración de la simulación."""
    date_picker = widgets.DatePicker(description='Fecha:', disabled=False)
    
    # Menú desplegable para seleccionar la hora inicial
    time_picker = widgets.Dropdown(
        options=[(f'{i}:00', i) for i in range(24)],
        value=6,
        description='Hora Inicial:',
        disabled=False,
    )
    
    # Slider para seleccionar la duración
    duration_slider = widgets.FloatSlider(
        value=12,
        min=1,
        max=24,
        step=1,
        description='Duración (h):',
        disabled=False,
    )

    # Función de actualización del gráfico
    def update_plot(date, start_hour, duration):
        if date is not None and start_hour is not None and duration is not None:
            start_datetime = datetime.combine(date, datetime.min.time()) + timedelta(hours=start_hour)
            start_datetime = timezone("America/Guayaquil").localize(start_datetime)
            plotSunAndPanelTrajectory(start_datetime, duration, start_hour)

    # Uso de widgets.interactive para actualizar automáticamente el gráfico
    interact = widgets.interactive(update_plot, date=date_picker, start_hour=time_picker, duration=duration_slider)

    # Mostrar los widgets
    display(interact)

if __name__ == "__main__":
    interactive_plot()

interactive(children=(DatePicker(value=None, description='Fecha:', step=1), Dropdown(description='Hora Inicial…